# Part 5: A Crayola Box Full of Colors
To recap, at 400 epochs we had 11 colors training to a pretty good loss of 0.01, and the neural network was predicting rather well when using the color sliders. Let's add a 24-color list to see what happens.

First we'll simply copy the network from the previous part. Click Run to create the train() function.

In [1]:
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential
import keras.optimizers, keras.utils, numpy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

def train(colorNameToRGBMap, epochs = 16):
    """
    Trains a neural network to understand how to map color names to RGB triples.
    The provided map is from 'color-name':(r,g,b) where r,g,b are floats in the range [0,1].
    Different names are allowed to map to the same RGB triple.
    Returns a trained model that can be used for recognize().
    """

    # Convert the Python map RGB values into a numpy array needed for training.
    rgbNumpyArray = numpy.array(list(colorNameToRGBMap.values()), numpy.float)
    
    # Convert the color labels into a one-hot feature array.
    # Text labels for each array position are in the classes_ list on the binarizer.
    labelBinarizer = LabelBinarizer()
    oneHotLabels = labelBinarizer.fit_transform(list(colorNameToRGBMap.keys()))
    numColors = len(labelBinarizer.classes_)
    colorLabels = labelBinarizer.classes_
    
    # Hyperparameters to define the network shape.
    numFullyConnectedPerceptrons = numColors * 16
    batchSize = 1
    
    model = Sequential([
        # Layer 1: Fully connected layer with ReLU activation.
        Dense(numFullyConnectedPerceptrons, activation='relu', kernel_initializer='TruncatedNormal', input_shape=(3,)),

        # Outputs: SoftMax activation to get probabilities by color.
        Dense(numColors, activation='softmax')
    ])

    print(model.summary())

    # Compile for categorization.
    model.compile(
        optimizer = keras.optimizers.SGD(lr = 0.01, momentum = 0.9, decay = 1e-6, nesterov = True),
        loss = 'categorical_crossentropy',
        metrics = [ 'accuracy' ])

    history = model.fit(rgbNumpyArray, oneHotLabels, epochs=epochs, batch_size=batchSize)

    return (model, colorLabels)

Using TensorFlow backend.


Let's consult the Internet to find a good list of colors matching a Crayola box. I found one great site [here](http://www.jennyscrayoncollection.com/2017/10/complete-list-of-current-crayola-crayon.html). Jenny lists the colors with 0-255 and hex color codes. To help translate that to floating point triples, let's create a small helper function to translate from 0-255 RGB to our floating point numbers, then create our map using that function and the colors in the 24-crayon box.

In [2]:
def rgbToFloat(r, g, b):  # r, g, b in 0-255 range
    return (float(r) / 255.0, float(g) / 255.0, float(b) / 255.0)

colorMap = {
    # 8-crayon box colors
    'red': rgbToFloat(238, 32, 77),
    'yellow': rgbToFloat(252, 232, 131),
    'blue': rgbToFloat(31, 117, 254),
    'brown': rgbToFloat(180, 103, 77),
    'orange': rgbToFloat(255, 117, 56),
    'green': rgbToFloat(28, 172, 20),
    'violet': rgbToFloat(146, 110, 174),
    'black': rgbToFloat(35, 35, 35),

    # Additional for 16-count box
    'red-violet': rgbToFloat(192, 68, 143),
    'red-orange': rgbToFloat(255, 117, 56),
    'yellow-green': rgbToFloat(197, 227, 132),
    'blue-violet': rgbToFloat(115, 102, 189),
    'carnation-pink': rgbToFloat(255, 170, 204),
    'yellow-orange': rgbToFloat(255, 182, 83),
    'blue-green': rgbToFloat(25, 158, 189),
    'white': rgbToFloat(237, 237, 237),

    # Additional for 24-count box
    'violet-red': rgbToFloat(247, 83 ,148),
    'apricot': rgbToFloat(253, 217, 181),
    'cerulean': rgbToFloat(29, 172, 214),
    'indigo': rgbToFloat(93, 118, 203),
    'scarlet': rgbToFloat(242, 40, 71),
    'green-yellow': rgbToFloat(240, 232, 145),
    'bluetiful': rgbToFloat(46, 80, 144),
    'gray': rgbToFloat(149, 145, 140),
}

(colorModel, colorLabels) = train(colorMap)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 384)               1536      
_________________________________________________________________
dense_2 (Dense)              (None, 24)                9240      
Total params: 10,776
Trainable params: 10,776
Non-trainable params: 0
_________________________________________________________________
None
Instructions for updating:
Use tf.cast instead.
Epoch 1/16
24/24 [==============================] - 0s 12ms/step - loss: 3.2501 - acc: 0.0000e+00
Epoch 2/16
24/24 [==============================] - 0s 2ms/step - loss: 3.2134 - acc: 0.0000e+00
Epoch 3/16
24/24 [==============================] - 0s 4ms/step - loss: 3.1919 - acc: 0.0000e+00
Epoch 4/16
24/24 [==============================] - 0s 4ms/step - loss: 3.1633 - acc: 0.0417
Epoch 5/16
24/24 [======

Wow, our loss is even higher than the last round with 11 colors, starting at 3.2 and only getting down to 2.6. We're going to have to repeat ourselves a lot more by using a higher epoch count. The epoch slider is below, with a max of 1000 epochs this time. See if you can find where the loss gets down to 0.01.

In [3]:
from ipywidgets import interact
@interact(epochs = (1, 1000))
def trainModel(epochs=100):
    global colorModel
    global colorLabels
    (colorModel, colorLabels) = train(colorMap, epochs=epochs)

interactive(children=(IntSlider(value=100, description='epochs', max=1000, min=1), Output()), _dom_classes=('w…

Did you see something strange? For me, at around 250 epochs the loss gets down to somewhere around 0.3, but then the loss starts bouncing around and not really going lower. In fact, sometimes it goes up! And it never gets any better; even at 1000 epochs it's still bouncing between 0.2 and 0.6.

Our neural network is stuck. Our teaching repetition is just making it confused.

Try training for 300 or so epochs, then try the color slider.

In [4]:
from IPython.core.display import display, HTML

def displayColor(r, g, b):
    rInt = min(255, max(0, int(r * 255.0)))
    gInt = min(255, max(0, int(g * 255.0)))
    bInt = min(255, max(0, int(b * 255.0)))
    hexColor = "#%02X%02X%02X" % (rInt, gInt, bInt)
    display(HTML('<div style="width: 50%; height: 50px; background: ' + hexColor + ';"></div>'))

@interact(r = (0.0, 1.0, 0.01), g = (0.0, 1.0, 0.01), b = (0.0, 1.0, 0.01))
def getPredictionsFromModel(r, g, b):
    testColor = numpy.array([ (r, g, b) ])
    predictions = colorModel.predict(testColor, verbose=0)  # Predictions shape (1, numColors)
    predictions *= 100.0
    print(colorLabels)
    numpy.set_printoptions(precision=1, suppress=True)
    print(predictions[0])
    displayColor(r, g, b)


interactive(children=(FloatSlider(value=0.5, description='r', max=1.0, step=0.01), FloatSlider(value=0.5, desc…

Are you having trouble seeing the best color match in that large list too? Let's fix it by sorting to show just the top 5 predictions.

In [5]:
numPredictionsToShow = 5
@interact(r = (0.0, 1.0, 0.01), g = (0.0, 1.0, 0.01), b = (0.0, 1.0, 0.01))
def getTopPredictionsFromModel(r, g, b):
    testColor = numpy.array([ (r, g, b) ])
    predictions = colorModel.predict(testColor, verbose=0)  # Predictions shape (1, numColors)
    predictions *= 100.0
    predColorTuples = []
    for i in range(0, len(colorLabels)):
        predColorTuples.append((predictions[0][i], colorLabels[i]))
    predAndNames = numpy.array(predColorTuples, dtype=[('pred', float), ('colorName', 'U50')])
    sorted = numpy.sort(predAndNames, order=['pred', 'colorName'])
    sorted = sorted[::-1]  # reverse rows to get highest on top
    for i in range(0, numPredictionsToShow):
        print("%2.1f" % sorted[i][0] + "%", sorted[i][1])
    displayColor(r, g, b)

interactive(children=(FloatSlider(value=0.5, description='r', max=1.0, step=0.01), FloatSlider(value=0.5, desc…

When I set it to about 500 epochs my loss was still pretty high, but the predictions when I moved the sliders were still pretty good, with the initial gray at 93% and in-between colors making sense.

### Coming up...
Can make the network need fewer epochs by using the color list to generate more colors?